In [46]:
# Step 1: Mount Google Drive and set paths
from google.colab import drive
drive.mount('/content/drive')

# 设置你的目录路径
my_path = '/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table'
dataset_path = f'{my_path}/dataset'

# Step 2: Clone YOLOv5 to the specified directory
!mkdir -p "{my_path}/yolov5"
!git clone https://github.com/ultralytics/yolov5.git "{my_path}/yolov5"
%cd "{my_path}/yolov5"
!pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path '/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5


In [47]:
%%writefile "{my_path}/yolov5/dataset.yaml"
train: "/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/train"
val: "/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/val"
test: "/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/test"  # 可选

nc: 16  # 总类别数量
names: ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'pingpong_table']


Overwriting /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/dataset.yaml


In [48]:

# Step 4: Ensure the dataset and labels exist
import os

def check_directory_exists(path):
    if not os.path.exists(path):
        print(f"Directory not found: {path}")
    else:
        print(f"Directory exists: {path}")

# 检查目录
train_images_dir = f"{dataset_path}/images/train"
train_labels_dir = f"{dataset_path}/labels/train"
val_images_dir = f"{dataset_path}/images/val"
val_labels_dir = f"{dataset_path}/labels/val"
test_images_dir = f"{dataset_path}/images/test"
test_labels_dir = f"{dataset_path}/labels/test"

check_directory_exists(train_images_dir)
check_directory_exists(train_labels_dir)
check_directory_exists(val_images_dir)
check_directory_exists(val_labels_dir)
check_directory_exists(test_images_dir)
check_directory_exists(test_labels_dir)

# 检查标签文件是否存在并修正无效标签
def check_label_classes(label_dir):
    invalid_labels = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file)) as f:
                for line in f:
                    class_id = int(line.split()[0])
                    if class_id != 0:
                        invalid_labels.append((label_file, class_id))
    return invalid_labels

def fix_labels(label_dir):
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file), 'r') as f:
                lines = f.readlines()
            with open(os.path.join(label_dir, label_file), 'w') as f:
                for line in lines:
                    parts = line.split()
                    class_id = int(parts[0])
                    if class_id != 0:
                        parts[0] = '0'
                    f.write(' '.join(parts) + '\n')

invalid_train_labels = check_label_classes(train_labels_dir)
invalid_val_labels = check_label_classes(val_labels_dir)

print("Invalid train labels:", invalid_train_labels)
print("Invalid val labels:", invalid_val_labels)

fix_labels(train_labels_dir)
fix_labels(val_labels_dir)

Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/train
Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/labels/train
Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/val
Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/labels/val
Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/images/test
Directory exists: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/dataset/labels/test
Invalid train labels: []
Invalid val labels: []


In [49]:

# Step 5: Train the model
!python train.py --img 640 --batch 16 --epochs 100 --data "{my_path}/yolov5/dataset.yaml" --cfg models/yolov5s.yaml --weights yolov5s.pt --name yolo9-pingpong-table --exist_ok



2024-06-23 10:57:18.566399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 10:57:18.566450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 10:57:18.567892: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=

In [56]:

# Step 6: Ensure the weights file is saved
weights_dir = f"{my_path}/yolov5/runs/train/yolo9-pingpong-table/weights"
print("Available weight files:", os.listdir(weights_dir))

# Step 7: Evaluate the model
weights_path = f"{weights_dir}/best.pt"
print("Using weights from:", weights_path)

!python val.py --weights "{weights_path}" --data "{my_path}/yolov5/dataset.yaml" --img 640 --task test


Available weight files: ['last.pt', 'best.pt']
Using weights from: /content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/runs/train/yolo9-pingpong-table/weights/best.pt
val: data=/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/dataset.yaml, weights=['/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/runs/train/yolo9-pingpong-table/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
git: 'Notebooks/robotic-vision/pingpong_table/yolov5' is not a git command. See 'git --help'.
YOLOv5 🚀 2024-6-23 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
test: Scanning /con

In [59]:
import cv2
import torch
import numpy as np
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.torch_utils import select_device
from yolov5.utils.augmentations import letterbox
from yolov5.utils.general import non_max_suppression

# scale_coords 手动实现
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    if ratio_pad is None:  # calculate from img0_shape and img1_shape
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    coords[:, :4] = coords[:, :4].round()
    return coords

# 加载模型
device = select_device('')
weights_path = "/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/yolov5/runs/train/yolo9-pingpong-table/weights/best.pt"
model = DetectMultiBackend(weights_path, device=device, dnn=False)

# 打开视频文件
cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/robotic-vision/pingpong_table/table.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 图像预处理
    img = letterbox(frame, 640, stride=32, auto=True)[0]  # 调整大小
    img = img.transpose((2, 0, 1))[::-1].copy()  # BGR to RGB, to 3x416x416 and copy
    img = torch.from_numpy(img).float().div(255.0).unsqueeze(0).to(device)

    # 推理
    pred = model(img, augment=False, visualize=False)
    pred = non_max_suppression(pred, 0.25, 0.45, classes=None, agnostic=False)

    # 处理推理结果
    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()

            # 绘制检测框
            for *xyxy, conf, cls in det:
                label = f'{model.names[int(cls)]} {conf:.2f}'
                cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 显示结果
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


YOLOv5 🚀 2024-6-23 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
